<h1><center> Computational Lexical Semantics</center></h1>
<h5><center>LIN5580-SEM1-A-1718</center></h5>
<h3><center>Angelo Basile and Jovana Urosevic</center></h3>
<h5><center>January 31, 2018</center></h5>

-----

### Instruction on how to use this notebook to replicate the results:

1. download and install python 2
2. download and install [pipenv](https://docs.pipenv.org/)
3. install cython separately: ```pipenv install cython```
4. install the requirements from the Pipfile
5. active the virtualenvironment: ```pipenv shell```
6. ```cd dissect-master```
7. install dissect: ```pipenv run python2 setup.py install```
8. deactive the virtual environment:```exit```
9. run the notebook: ```pipenv run jupyter notebook```

#### Extra:

For the POS-tagging tasks, it is required to download language models.

- ```pipen run python2 -m spacy download en``` for downloading the English model
- ```pipen run python2 -m spacy download es``` for downloading the Spanish model

In [1]:
# confirm that dissect is installed correctly:
# this import should give no errors
from composes.semantic_space.space import Space

___________

#### 1) Describe the Distributional Hypothesis and explain how it can be used to find semantically similar words in corpora. (3 points)

Distributional hypothesis was introduced by Harris (1968) and refers to the relation between the word meaning and the context it appears in (that is, the distribution of other words that accompany it and/or their syntactic relations) (Jurafsky and Martin, 2017). If we can understand the word meaning from the context it is commonly used in, then the words with similar meaning should be very likely to appear in similar contexts (ibid.).  
&ensp;&ensp; In order to see how this works in practice, we can look at the example below:

(1) &ensp;*The woolly **lemur**, the aye aye and the ring tail **lemur** can be found on the island of Madagascar.  
&ensp;&ensp;&ensp; **Lemurs** are best known for their large, round reflective eyes and their wailing screams.  
&ensp;&ensp;&ensp; The biggest threat to the **lemur** is deforestation.*  

If we did not know what the word *lemur* was, just based on these sentences we could find out that it is an animal that lives on Madagascar, that has big, round eyes and loud screams, that it probably lives in the trees, etc. Besides this, if we take a look at the concordance view in the corpus (example 2), not only will we see other contexts that explain the word *lemur*, but also other similar words such as monkey, primate, species, etc. that frequently appear with it.

(2)
*&ensp;&ensp; -xxnigeoo-langren-20101250the lemur **monkey** or &ensp;&ensp;&ensp;**lemur** &ensp;&ensp;&ensp;&ensp;&ensp; is surely an unique **animal** that is native  
&ensp;&ensp;&ensp;&ensp; forests, and provides homes for red ruffed &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; &ensp; **lemur** &ensp;&ensp;&ensp;&ensp;&ensp; , **gorillas, colobus monkeys**, and **jaguars**  
&ensp;&ensp;&ensp;&ensp;&ensp; Concreting the area in front of the new &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **lemur** &ensp;&ensp;&ensp;&ensp;&ensp; enclosure we could have enough done over  
&ensp;&ensp;&ensp;&ensp; so now at gymnastics my ten year old is a &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;   **lemur** &ensp;&ensp;&ensp;&ensp;&ensp; leaping a **cheetah** chasing a **spider** spanning  
&ensp;&ensp;&ensp;&ensp; **elephants, tigers (cubs), zebras, snakes**,  &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **lemurs** &ensp;&ensp;&ensp;&ensp; **monkeys, birds, kangaroos, ostriches**,  
&ensp;&ensp;&ensp;&ensp; Really feel distrusted and alienated The &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; &ensp;   **lemur** &ensp;&ensp;&ensp;&ensp;&ensp; **monkey** is called so simply because it is  
&ensp;&ensp;&ensp;&ensp; Live here including 90% of the world’s the &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;   **lemur** &ensp;&ensp;&ensp;&ensp;&ensp; **monkeys** . Madagascar is made up of*  

The fact that we can frequently see other words naming types of animals around the word *lemur*, points to a kind of a relation these words have with our word *lemur*. And if we look the word *gorilla* up in the corpus (example 3), we will again get quite similar words around it in the context when it is used to refer to an animal (and not as an attribute for a person): *monkey, chimpanzee, animal, species,* with the description of where and how it lives, etc. So quite similar to the previous example. And this is what distributional hypothesis is about - we can understand the meaning of a word from the context, and we can find other similar words by looking at other similar contexts (since gorilla and lemur have similar meaning, they will appear in similar contexts).

(3)*&ensp; to Goodison and hearing those **animals** doing &ensp;&ensp;&ensp;&ensp;&ensp;**gorilla** &ensp;&ensp;&ensp;&ensp; impressions than anything else Ive ever heard.  
&ensp;&ensp;&ensp; Important all-year-round foods of the lowland &ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **gorilla** &ensp;&ensp;&ensp;&ensp; in Gabon include the large ground-living  
&ensp;&ensp;&ensp; of the feeding behaviour of the lowland &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **gorilla** &ensp;&ensp;&ensp;&ensp; and its interaction with other fruit-eating  
&ensp;&ensp;&ensp; the near relatives of man, the **chimpanzee** and &ensp;&ensp;&ensp;&ensp;**gorilla** &ensp;&ensp;&ensp;&ensp; . In the case of these **species**, rigidly  
&ensp;&ensp;&ensp; evolutionary  ties with thee **chimpanzee** and &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;**gorilla** &ensp;&ensp;&ensp;&ensp; , but also his rather less obvious resemblance  
&ensp;&ensp;&ensp; relative, the **chimpanzee**, and even more in the &ensp;&ensp;&ensp; **gorilla** &ensp;&ensp;&ensp;&ensp; . But man is, in general, sexually dimorphic in  
&ensp;&ensp;&ensp; are  quite unlike anything found in the &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **gorilla** &ensp;&ensp;&ensp;&ensp; or **chimpanzee**. They are, however, closely*  
<br>
&ensp;&ensp; In fact, in computational semantics, we can build distributional semantic models to calculate similarity between certain words by comparing the contexts they appear in (Jurafsky and Martin, 2017). The idea is that the more contexts two words share, the more similar they are. In DSMs, contexts are represented as vectors of co-occurrence counts denoting how often words co-occur with one another (ibid.). Their similarity is then calculated by comparing these vectors: if two words have similar context vectors, then they have similar meaning (Basili and Pennacchiotti, 2010; Jurafsky and Martin, 2017). These are high dimensional, sparse vectors (containing many zeros) because most of the words will just not happen to appear in the context of other words. This vector representation of a word meaning is called embedding, while the models trying to use words represented in this way are called *vector space models, word space models*,  or *vector based models* (Van der Plas, 2017; Jurafsky and Martin, 2017).  
&ensp;&ensp; In order to calculate the similarity between terms, we can consider the following example.

|          | get  |  see | use  | hear |
|:--------:|:----:|:----:|:----:|:----:|
| lemur    |  5   |  35  |  2   |  22  |
| gorilla  |  9   |  65  |  3   |  36  |
| cat      |  35  |  74  |  5   |  40  |
| chair    |  81  |  35  | 104  |   0  | 
<h6 style="text-align: center;" markdown="1">Table 1. Term-context matrix.</h6>    

In the table we see a hypothetical example of co-occurrence counts for the target words (raws) and their context words (columns). In order to calculate the similarity between the target words given that they appear in the context of the words *get* and *see*, we present them in a n-dimensional (Euclidean) space (Evert, et al., 2010). The position of the words is determined by their co-occurrence counts with the words *see* and *get*: *lemur = (10, 76)*.   
![caption](pics/figure1.jpg)
<h6 style="text-align: center;" markdown="1">Figure 1. Target words as points (left) or vectors (right) in two-dimensional space.</h6>    

There are several ways of counting similarity between the words now: by either counting the Euclidean distance between the points (spatial proximity), or by measuring the angle between the vectors (ibid.). Measuring the angle is a better option because it takes into account vector’s direction rather than position of the word. For example, if we take a look at the distance of the points in the figure 1 (left), *cat* and *gorilla* will seem more similar than *gorilla* and *lemur*. However, if we look at the angle between the vectors on the right on the figure 1, we will notice that it is very small between *gorilla* and *lemur*, showing their very close similarity, as compared to the angle between *gorilla* and *cat*, or in that matter *chair*.

#### 2) Because DSMs generate ranked lists of semantically similar words, they can be seen as alternatives to manually built lexical resources such as WordNet. (5 points)
**- Explain the main differences between the lexical semantic content of WordNet and the lexical output of DSMs.  **  
WordNet is a type of a thesaurus in which not only can we find words’ definitions and their synonyms, but also other words they are related to by a certain semantic relation (synonymy, hyponymy, polysemy, metonymy, meronymy / holonymy, antonymy) (Miller et al, 1990). It contains open class words (nouns, adjectives, verbs and adverbs) which are organized by their senses and grouped into synonym sets (synsets). Besides, all words are hierarchically structured in order to represent the semantic relations between them (ibid.).  
&ensp;&ensp; Compared to the DSMs, when using WordNet it is relatively easy to extract semantic relations between a target word and context words since that information is explicitly represented in its structure. On the other hand, when it comes to the output of a DSMs for a specific target word, we get only a sorted list of most semantically related words together with their scores, but without any information about what kind of relation exists between the target word and the outputed words.  
<br>
**- Discuss advantages and drawbacks of manually built lexical resources such as Wordnet and automatically generated lexical resources such as lists of semantically similar words stemming from DSMs. **  
When it comes to WordNet, one of its advantages is that it is hand-built by linguist and therefore very precise (Van der Plas, 2017). However, this kind of resource is expensive and time-consuming to create and can also be quite difficult to maintain and keep updating for new words. These are the reasons why it is difficult to have a similar resource for other languages (ibid). Furthermore, it is definitely an advantage to have explicit relations marked between different senses of words together with glosses explaining the senses. One of the improvements here could be to provide currently missing frequency information for the synsets. If we take a look at WordNet, we will see that a lot of senses following the first ones are quite infrequent in English language, but no explicit information is given about this (ibid.). Also, when it comes to relations between the synsets, most of them are represented as binary, which is not the case for synonyms for example; there is a higher or lower level of similarity / relatedness that is not quantified in WordNet (ibid.). Here, words are either synonyms or not. This, however, can be overcome using a thesaurus-based algorithms to calculate words similarity given the hierarchical structure of WordNet, in which case the similarity is calculated based on the distance of two words in the hierarchy (Jurafsky and Martin, 2017).  
&ensp;&ensp; On the other hand, DSMs have their pros and cons as well. These models are fast and inexpensive to obtain as opposed to WordNet creation, and therefore can be applied to any domain and any language (Van der Plas, 2017). Also, they provides a way to quantify semantic relatedness between words (ibid.). However, the significant disadvantage DSMs have compared to WordNet is that besides the similarity scores we get in the output, we have no further information about the relations between observed words. We do not get only synonyms in the outputs, but all other sorts of semantic relations. Similarity in this approach is taken quite broad and has as its disadvantage not being able to distinguish between different relations, such as  synonymy, antonymy, etc. (Sahlgren, 2001).   
<br>
**- Can you think of an application that would fare better when using WordNet and another that would benefit more from the output of DSMs?**  
WordNet is commonly used for the task of word sense disambiguation, where we need to find the most appropriate sense of a word given the context it appears in (Morato et al., 2004). However, it could be interesting to use WordNet in a task involving inference. For example, given the sentences *Johnny is sleeping* and *Johnny is snoring*, we can predict entailment based on the hypernym-hyponym relation of *snoring* and *sleeping* since we can say that the former entails the latter.  
&ensp;&ensp; DSMs could be used in many applications, such as in information retrieval, where we can retrieve documents containing similar words to the ones in a query. Also, it could be used in machine translation, where DSMs could help us to find out if two words are similar enough to be used in the same context, etc. (Jurafsky and Martin, 2017).


-------
#### In this assignment, in order to gather semantically similar words, we will use the following:
**1. Data:**
- A monolingual corpus: **IMDB English language corpus**, available [here](https://github.com/explosion/thinc/blob/master/thinc/extra/datasets.py) and extracted using spaCy library  
- Word-aligned parallel texts for a language pair **Europarl English-Spanish corpus**, available [here](http://opus.nlpl.eu/download/Europarl/en-es/)

**2. Tools:**
- an extraction program (for the extraction from the parallel text): a program that gathers coocurrence counts for the target words
- a DSM tool, a program that will compute the similarity between the coocurrence vectors for the head words we are interested in – taking the coocurrence matrix as input
------------

#### Building a coocurrence matrix from a corpus:
There are several ways to build a coocurrence matrix depending on the target words you are selecting (parameter 1) and the type of features, more in particular, the type of context you are selecting (parameter 2). Please refer back to the slides of the lecture on DSMs for a list of parameters and a description of the options.

#### *TARGET WORDS*:

#### 3) Does the monolingual corpus you are using to extract the cooccurrence counts from provide lemmas? Discuss why using lemmas instead of words often leads to better performances in DSMs. (3 points)
The monolingual corpus we are using does not provide lemmas. However, the lemmatization should be applied since it is a type of text normalization that will help us reduce dimensionality, that is, avoid data sparseness (Turney and Pantel, 2010). Here  we are interested in word similarity, and therefore we will want to convert all the different grammatical forms of a certain word into one (lemma), since they all carry essentially the same meaning. In this way, we will avoid counting separately in the co-occurrence matrix different forms of the same word. Lemmatization normally leads to a better performance because in this way we are eliminating redundant grammatical information, while making it easier to recognize semantic similarities among words (ibid.).

-----------------------------------------------------------------

#### 4) Extra points: The parallel text does not provide lemma information, nor PoS tags. Run a PoS tagger on (both sides of) the parallel text and use the information it gives you for the DSM. (+5 points)

**The data provided with DISSECT already selected target words for you. For the parallel text, and in case you selected your own monolingual text, we take the 1550 most frequent content words (lemmas) in the corpus as target words. In order to determine the 1550 most frequent content words in either corpus, you will need to count all content words. (If you do not have access to PoS information, use a list of stopwords that you filter out.) You then need to rank these words according to their frequency and take the top 1550.**

#### 5) Explain why PoS information can be useful for distinguishing between ambiguous terms. Give some examples from the data you are using. (3 points)
POS information can be useful because it helps us disambiguate between formally identical words with different meanings (Turney and Pantel, 2010). This does not refer to homonyms, but rather words that have the same form only in certain contexts and are distinguishable by their part-of-speech. For example, the word *work* (noun,verb) without POS information could be translated to Spanish as *trabajo* (noun) or *trabajar* (verb). If we apply a POS-tagger on our data, we avoid having this type of issue.  
&ensp;&ensp; Since we are working with a parallel corpus, not distinguishing between different POS can mislead us into extracting wrong types of translations into our co-occurrence matrix. The following examples are from the Europarl English-Spanish parallel corpus that we are using. The Spanish word *poco* can have different translations into English depending on its part-of-speech (example 4).

(4) &ensp; *(Spanish: determiner):&ensp;&ensp;	tienen dificultades con esta cuestión a dar esos **pocos** pasos más*  
&ensp;&ensp;&ensp;&ensp;  *(English: adjective):&ensp;&ensp;&ensp;&ensp; who have difficulties in this matter to take those **few** extra steps*  
&ensp;&ensp;&ensp;&ensp;  *(Spanish: adverb):&ensp;&ensp;&ensp;&ensp;&ensp; Esto me molestó un **poco** porque*   
&ensp;&ensp;&ensp;&ensp;  *(English: adverb):&ensp;&ensp;&ensp;&ensp;&ensp; I was **somewhat** irritated about this* (informally could be also translated as **a bit, a little**)  
<br>
In the opposite case, we can have English word *well* differently translated depending on it POS information (example 5).  

(5) *&ensp; (English: adverb):&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; this work has been carried out extremely **well**  
&ensp;&ensp;&ensp;&ensp;  (Spanish: adverb):&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; Creo que es un trabajo muy **bien** realizado  
&ensp;&ensp;&ensp;&ensp;  (English: interjection):&ensp;&ensp;&ensp; **Well**, we in Britain are looking to the European Court  
&ensp;&ensp;&ensp;&ensp;  (Spanish: interjection):&ensp;&ensp;&ensp; **En fin** , en Gran Bretaña hemos recurrido   
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;**Bien**, es verdad que se redactan regulaciones* (informally, it could also be also translated as **vaya, hala**)  
*&ensp;&ensp;&ensp;&ensp; (English: noun):&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; is like carrying water to fill up a dry **well**  
&ensp;&ensp;&ensp;&ensp; (Spanish: noun)&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;es como si llevásemos agua al **pozo**  *

---------------------------
#### *FEATURES*:

#### 6) Please discuss in your report, the different types of features, more in particular, the types of context (parameter 2) that are employed for DSMs in general. (5 points)
As previously shown, DSMs are based on a co-occurrence matrix. There are several types of co-occurrence matrices that we could build, such as term-document matrix, term-context matrix and pair-pattern matrix (Turney and Pantel, 2010; Jurafsky and Martin, 2017).  
&ensp;&ensp; **1. Term-document matrix.** In this kind of matrix rows represent terms (words of the vocabulary), columns represent documents (they can be actual documents of some kind or just a bigger collection of text like web pages), while cells contain the number of times each target word appears in each document (Jurafsky and Martin, 2017). In this way, we have every document represented as a count vector or as a point in a n-dimensional space. Two documents are considered to be similar if they contain similar words, and therefore have similar vectors (ibid.). As seen earlier, we can measure the similarity between the documents either by measuring the distance between the points or by measuring the angle between the vectors in the semantic space. The smaller the angle or distance, the more similar two documents are. In other words, the more similar words they contain, the more similar they are.   

|          |  doc1  |  doc2  |  doc3  |     
|:--------:|:------:|:------:|:------:|    
| lemur    |   0    |   15   |    1   |
| gorilla  |   0    |   21   |    2   |
| cat      |   6    |    4   |   17   |
| chair    |   12   |    0   |    0   |

<h6 style="text-align: center;" markdown="1">Table 2. Term-document matrix.</h6>    
<br>
&ensp;&ensp; **2. Term-context matrix** (also called word-word or term-term matrix). When it comes to term-context matrix, the rows of a matrix are again target words (terms), while in the columns we have a narrower context than previously used document, however it can refer to anything from a whole vocabulary, certain words, paragraphs, sentences, to specific syntactic relations among words (Evert et al., 2010; Van der Plas, 2017). The following subsections summarize all *context* possibilities.  
<br>
&ensp;&ensp; ** - Surface context: **  
This type of context refers to the “window” around the target word, which basically consists of n words or characters to its left and n words or characters to its right (ibid.). In this case, the cells in the matrix contain counts of how many times the word in the column appeared in the n-word window around the row (target) word. For example, if we have the 7-word window around the words *gorilla, lemur, cat* and *chair*, we get the following example sentences (obtained from the British National Corpus): 

(6)*&ensp; with the consequent loss of habitat for &ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **gorillas** &ensp;&ensp;, elephants and other threatened animal  
&ensp;&ensp;&ensp;&ensp; that there is a home for owls and squirrels, &ensp;&ensp; **lemurs** &ensp;&ensp;, parrots and toucans. A river with slightly  
&ensp;&ensp;&ensp;&ensp; outback have been set back by domestic &ensp;&ensp;&ensp; **cats** &ensp;&ensp;&ensp;&ensp; , which have killed 11 of the rare animals  
&ensp;&ensp;&ensp;&ensp; Kendal would you like to bring your &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **chair** &ensp;&ensp;&ensp;&ensp; over. Rachael can you see the board from where*  

Then, we can make a matrix (table 3) with the four target words in rows and some exemplar context words in columns extracted from the 7-word window (there are many more words that could be added to the matrix, but this is just an example).  

|           | ... |  habitat  |  elephants | animal | squirrels| board | ... |
|:---------:|:---:|:---------:|:----------:|:------:|:--------:|:-----:|:---:|
| lemurs    | ... |     0     |      0     |   0    |     1    |   0   | ... |
| gorillas  | ... |     1     |      1     |   1    |     0    |   0   | ... |
| cats      | ... |     0     |      0     |   1    |     0    |   0   | ... |
| chair     | ... |     0     |      0     |   0    |     0    |   1   | ... |

<h6 style="text-align: center;" markdown="1">Table 3. Term-context matrix.</h6>    
&ensp;&ensp;The size of the window can be as small or as big as we need. Depending on what kind of relations we want to extract, we will need one or the other (Evert et al., 2010). More about how the window size can influence the relations we can extract from corpus is in the question 7. Furthermore, we can choose if we want to get both the left and the right context, or only one of them, depending on if we are interested in the words that follow or precede the target word, or both. For example, if we wanted to see what kind of adjectives normally describe a lemma *woman*, then in that case we would want to extract only the left context. And finally, we can chose to weight words that are closer or further from the target word in the context, again depending on what kind of information we want to extract; we can have either uniform or distance-based weightening (Evert et al., 2010).   
<br>
&ensp;&ensp; ** - Textual context:**   
This type of context refers to the fact that the context terms belong to the same linguistic unit as the target term (Evert et al., 2010). Commonly these linguistic units are sentences or paragraphs, but can also be turns in a conversation or texts on web pages (Evert et al., 2010; Van der Plas, 2017). This kind of context somewhat resolves the problem surface context has: we need to choose a particular, fixed number as a window size (Evert, 2008). This will not always fulfill our expectations: window can sometimes be too big, and go over sentence or paragraph boundaries, or too small and not catch the context we want. For example, catching a context of *throw* in a 3-word window will catch *throw a birthday party*, but not *throw a big birthday party* (Evert, 2008). This can be particularly problematic when working with a language that has a more free word order and where closely related words could appear far apart in the context. In those cases, textual context would be more appropriate to use.  
<br>
&ensp;&ensp; **- Syntactic context:**  
This type of context includes co-occurrence counts of the target words and the words that have a specific syntactic relation with the target words (Evert et al., 2010; Van der Plas, 2017). We can have different types of relations such as subject, object, modifier, coordination, apposition, etc. (ibid.). In this way, the columns of the matrix are syntactic relations with specific words we are interested in, while the cells contain how many times the target and the context word appeared in a particular syntactic context (table 4). As compared to the surface context, it does not set a strict word limit as context but it manages to catch relations happening further away from the target word; while compared to the textual context, it introduces less noise (Evert, 2008).  

|          |  eat-subj |  paint-obj |  long-tailed-adj | have-obj |   
|:--------:|:---------:|:----------:|:----------------:|:--------:|  
| lemur    |     32    |      0     |         43       |    16    |
| gorilla  |     35    |      0     |          9       |    17    |
| cat      |     30    |      0     |          5       |    25    |
| chair    |      0    |     20     |          0       |    24    |

<h6 style="text-align: center;" markdown="1">Table 4. Term-context matrix: syntactic context.</h6>    
<br>
&ensp;&ensp; **- Multilingual or translational context**  
When it comes to multilingual context, it can contain as columns target word translated to other languages, while cells contain how many times a certain target word was translated in a particular way in a particular language (Van der Plas, 2017). This kind of information we can extract from aligned parallel corpora. For example, in the table 5 we can see how many times the formal variations with the same meaning, *autumn* and *fall*, were translated as such in Italian and Dutch. Besides looking at how one word is translated across languages, we can also look at how a certain word or phrase is translated from one language to another. An example for that would be the Spanish word *poco* that can be translated to English as *a few, few, a little, little* or *a bit*. We can check, for example, which of these translations is the most frequent.  

|          |  autunno-IT |  herfst-NL | 
|:--------:|:-----------:|:----------:|
| fall     |       8     |     13     |
| autumn   |      10     |      8     |  

<h6 style="text-align: center;" markdown="1">Table 5. Term-context matrix: translational context.</h6>    
<br>
&ensp;&ensp; **3. Pair-pattern matrix.** The pair-pattern matrix is used to measure similarity between different kind of patterns (Turney and Pantel, 2010). Here the columns represent the patterns, for example *X solves Y*, while the cells contain the counts of how many times target words appear as a part of this pattern. As stated by Lin and Pantel (2001) (in Turney and Pantel, 2010), pairs of target words that co-occur with similar patterns, tend to have similar meaning. The idea is that given the pattern *X solves Y*, we will find similar patterns such as *Y is solved by X, Y is resolved in X*, and *X resolves Y* (Turney and Pantel, 2010). That is, these patterns tend to appear together with similar X and Y word pairs, which points to their similar meaning. For example, pairs *mason : stone, carpenter : wood* appear with similar semantic relation *artisan : material*  (Turney and Pantel, 2010).  

**For the aligned parallel text, we are going to select the words that are aligned to the target word as features. Please select only the 10000 most frequent content words as features. If you opted to include your own monolingual corpus, we are going to select a small window of the content word following and the content word preceding the target word, as features. Please select only the 10000 most frequent content words as features.**

#### 7) Discuss what influence the size of the window has on the nature of the semantic relations we find between target word and semantically similar words proposed by the system. What consequences do you think a larger window size would have on the semantically similar words proposed by the system? (3 points)
Example from Evert et al. (2010), nearest neighbours for a word *car* extracted from BNC with different window sizes:  

(7) &ensp;2-word window: *van,  vehicle, truck, motorcycle, driver, motor, lorry, motorist, cavalier*  
&ensp;&ensp;&ensp;&ensp;30-word window: *drive, park, bonnet, windscreen, hatchback, headlight, jaguar, garage, cavalier*    
<br>
&ensp;&ensp;In general, the shorter the window, the more syntactic information we catch since we are focusing only on the nearest neighbours of the target word, while on the other hand, longer window catches more semantic relations between the words (Jurafsky and Martin, 2017). In this sense, we have two kinds of similarity as stated by Jurafsky and Martin (2017). Firstly, we say that words have first-order co-occurrence (syntagmatic association) when they are normally found near each other. For example, that is the case with words *van* and *vehicle* with the verb *drive*; they indeed normally appear close to one another with *van* and *vehicle* as objects of the verb *drive*. On the other hand, we have a second-order co-occurrence (paradigmatic association) if the words have similar neighbours (Jurafsky and Martin, 2017). For example, this is the case with the word *car* and words *windscreen*, *bonnet*, etc.  
&ensp;&ensp; When it comes to the type of a semantic relation depending on the window size, we can see in the example 7 that in the shorter window we catch mostly co-hyponyms as the most similar words. Which is no surprise, because co-hyponyms will tend to appear in a similar syntactic context as the target word *car*. On the other hand, with a wider window, we catch  words denoting parts or types of a car as the most similar words. This is because the wider the window, the more of a context “topic” we are catching and therefore most common further neighbours of a *car*. 

#### 8) Discuss the differences in output of the DSM (in terms of the similar words it will produce) you expect between the two types of input you are using (the monolingual corpus and the aligned parallel files). Which type of data do you think will generate the highest percentage of synonyms amongst the nearest neighbours? (3 points)
It is expected that the multilingual DSM will generate higher number of synonyms as opposed to the monolingual one. The reason for this is that in the parallel data we are looking at the aligned translations, and therefore will extract more translational variations of the target words. For example, if working with English as one of the languages, for the nearest neighbours of the word *bicycle* we will expect to get in the output words such as *bike, cycle, two-wheeler*, etc. (Van der Plas, 2017). However, the monolingual corpus is expected to extract other types of semantic relations among outputted neighbours besides synonyms, such as co-hyponyms, antonyms, hyponyms, meronyms, etc. This is because for the monolingual DSM we are using the whole corpus as context, and therefore we can expect the output to resamble something similar to a topic analysis.

--------------------
#### 9) Write a python program that extracts coocurrence counts for all target words and features from the parallel aligned files (and the monolingual corpus, if you are using a different corpus than the one provided by DISSECT).  Include the extraction program you wrote in the submission, and paste 20 lines (for 20 target words) of the coocurrence counts you extracted in the report. The output of your program should be formatted according to the input format the DSM tool you are using requires. (36 points)

In [ ]:
# EXTRACTION PROGRAM
# monolingual
import spacy
import codecs
from __future__ import print_function
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to tag the text
def tag(token):
    word = token.text
    tag = token.tag_
    return word+'_'+tag

# load the imbdb corpus
import thinc.extra.datasets
data, _ = thinc.extra.datasets.imdb()
texts, _ = zip(*data[-10000:])

# filter the text based on POS
filtered = []
for doc in texts:
    filtered.append(' '.join(w.text for w in nlp(doc) if w.pos_ in ['NOUN', 'VERB']))

# build the frequency distribution for words
from collections import Counter
word_freq = Counter()
for t in filtered:
    word_freq.update(t.split())

# use only the 1550 most frequent words
vocab = [x[0] for x in word_freq.most_common(1550)]
assert len(vocab) == 1550

#build the wordXword matrix
from sklearn.feature_extraction.text import CountVectorizer
docs = filtered
count_model = CountVectorizer(vocabulary=vocab)
X = count_model.fit_transform(docs)
Xc = (X.T * X) 
Xc.setdiag(0)

# invert the vocabulary
di = dict([[v,k] for k,v in count_model.vocabulary_.items()])

# save the output in a dissect compatible format
with open('mono.dm',"w") as o:
    for word, counts in zip(vocab, Xc.toarray()):
        o.write(word.encode('utf-8')+' '+" ".join(map(str, counts)))
        o.write('\n')

with open('mono.rows', 'w') as o:
    for w in vocab:
        o.write(w.encode('utf-8'))
        o.write('\n')

# note: this is the same as before, since we are using all the words as context
with open('mono.cols', 'w') as o:
    for w in vocab:
        o.write(w.encode('utf-8'))
        o.write('\n')

# BUILD THE DSM
from composes.semantic_space.space import Space

#create a space from co-occurrence counts in dense format
mono = Space.build(data = "./mono.dm",
                   rows = "./mono.rows",
                   cols = "./mono.cols",
                   format = "dm")
# extract similar words
from composes.similarity.cos import CosSimilarity
# 5 most common
for w in word_freq.most_common(5):
    print(mono.get_neighbours(w[0], 5, CosSimilarity()))
    
# 5 least common
for w in word_freq.most_common(1500)[-5:]:
    print(mono.get_neighbours(w[0], 5, CosSimilarity()))
    
# apply a transformation
# transformation
from composes.transformation.scaling.ppmi_weighting import PpmiWeighting
transformed = mono.apply(PpmiWeighting())

import pprint
for w in word_freq.most_common(5):
    pprint.pprint(transformed.get_neighbours(w[0], 5, CosSimilarity()))

In [ ]:
=======CODE FOR PARALLEL SECTION, QUESTION 9)======
import pandas as pd
import codecs
with codecs.open('./data/c.clean.en/data', 'r', encoding='utf-8') as f:
    en = f.read().lower().splitlines()

with codecs.open('./data/c.clean.es/data', 'r', encoding='utf-8') as f:
    es = f.read().lower().splitlines()

with open('./data/en-es_aligned.intersect/data', 'r') as f:
    align = f.read().splitlines()
    
# check shapes
assert len(en) == len(es)
assert len(en) == len(align)
print(len(en))

en_lemmas = []
es_lemmas = []
for doc in en:
    en_lemmas.append(' '.join(w.lemma_ for w in nlp(doc)))
    

for doc in es:
    es_lemmas.append(' '.join(w.lemma_ for w in nlp(doc)))

# filter the text based on POS
en_tagged = []
es_tagged = []
for doc in en:
    en_tagged.append(' '.join(w.text+'_'+w.pos_ for w in nlp(doc)))
for doc in es:
    en_tagged.append(' '.join(w.text+'_'+w.pos_ for w in nlp(doc)))

# build the frequency distribution for words
from collections import Counter
word_freq = Counter()
for t in filtered:
    word_freq.update(t.split())

# use only the 1550 most frequent words
vocab = [x[0] for x in word_freq.most_common(1550)]
assert len(vocab) == 1550

corpus = zip(en, es, align)
assert len(corpus) == len(en)
len(corpus)

from collections import defaultdict
mm = defaultdict(int)

for x,y,z in corpus:
    zs = z.split(' ')
    for words in zs:
        try:
            mm[x.split()[int(words.split('-')[0])],
               y.split()[int(words.split('-')[1])]] += 1
        except:
            pass

en = []
es = []
count = []
for k,v in mm.items():
    en.append(k[0])
    es.append(k[1])
    count.append(v)

assert len(en) == len(es)
assert len(en) == len(count)

sm = pd.DataFrame([en,es,count]).T
print(sm.head())

sm.to_csv('parallel.sm', index=False, sep=' ', header=False)

with open('parallel.rows',"w") as o:
    for word in en:
        o.write(word)
        o.write('\n')

with open('parallel.cols',"w") as o:
    for word in es:
        o.write(word)
        o.write('\n')

# parallel
from composes.semantic_space.space import Space

#create a space from co-occurrence counts in sparse format
parallel = Space.build(data = "./parallel.sm",
                       rows = "./parallel.cols",
                       cols = "./parallel.rows",
                       format = "sm")

# build the frequency distribution for words
from collections import Counter
wf = Counter()
for t in es:
    wf.update(t.split())

wf.most_common(5)

# 5 most common
for w in wf.most_common(5):
    try:
        pprint.pprint(parallel.get_neighbours(w[0], 5, CosSimilarity()))
    except:
        pass

----------------
#### *Feature weighting and similarity function:*

#### 10) Explain what function the feature weighting scheme has. What would be the drawback of doing without a weighting scheme and using raw frequencies instead? (3 points)
When working with language and having a big collection of words as features, we commonly end up with having a very noisy feature representation, which means that we have a lot of features that are redundant and not very informative for the task we are trying to conduct (Jurafsky and Martin, 2017). The purpose of using a measure of feature weighting is to reassign new counts to the co-occurrence matrix, but this time taking into account how informative they are for our task. For example, if we just take raw counts into consideration, it will turn out that frequent words, such as *the, a it, about, on, for*, etc. (stop words), appear very frequently in a lot of contexts and therefore mislead us into thinking that they are similar with a lot of different kinds of words just because they appear very frequently. So the raw counts are not the best measure of similarity between words and can many times be misleading (Jurafsky and Martin, 2017; Van der Plas, 2017). In order to extract the most informative context words, we need to measure how informative they are.   
&ensp;&ensp; There are several weightnening measures we can use, for example Pointwise mutual information or log-likelihood ratio. These are association measures of “how much more often  than chance two words co-occur” (Jurafsky and Martin, 2017). In order words, they weigh the association between the target word and the features by comparing the expected and the observed co-occurrence frequency (Van der Plas, 2017). The less frequent the target word and the context word are, the higher weight is given to their observed co-occurrence count  (because their expected chance co-occurrence frequency is low) (Evert et al., 2010).    
&ensp;&ensp; Another very known feature weighting method is tf-idf. It is not used much when measuring semantic similarity, but it is used in many other applications, especially information retrieval where it is a dominant method for weighting frequencies. (Jurafsky and Martin, 2010). What it does is that it gives less weight to the features that appear in a lot of different documents, because that means they are not very informative for distinguishing between different types of documents. Consequently, more weight is given to the words that do not appear as often and appear in only a certain type of document, since it shows that they are a good indicator of that document type (ibid.). So more weight is given to a term frequent in a certain document type, but rare in general in other documents.

#### 11) List the weighting schemes the tool you are using includes and what similarity functions. Make your choice for the weighting scheme and similarity function you will use and motivate your choice in the report. (3 points).
The tool we are using is DISSECT and it has the following weighting schemes: Positive Pointwise Mutual Information, Positive Local Mutual Information, Exponential Pointwise Mutual Information, and Positive Log Weighting. The similarity measures it offers are Cosine similarity, Dot product, Euclidean similarity, and Lin similarity.  
&ensp;&ensp; For the purpose of our task we decided to use Positive Pointwise Mutual Information as a weighting scheme and Cosine similarity as a similarity function. The choice was made according to the fact that these are very known and widely used methods in NLP literature. Besides, these methods were covered during the lectures and therefore we had confidence to be able to implement them and interpret the results obtained in a proper manner.

---------------
#### *Analysing the output:*

**Now we are ready to generate some output. You should be able to get the distributional similar words of any of the 1550 target words you selected.**

#### 12) For both types of corpora (multilingual and monolingual), retrieve the top-5 semantically similar words for five high-frequency words and five low-frequency words from the 1550 words. Paste the output in the report. (5 points) 
Monolingual corpus, the most frequent words:
```
[('is', 1.0000000000000002),
 ('has', 0.6101378493796382),
 ('man', 0.591261237239224),
 ('takes', 0.5842141528025531),
 ('are', 0.568289728777084)]
[('was', 0.9999999999999999),
 ('had', 0.726944791320728),
 ('were', 0.7118683378473183),
 ('did', 0.6819094209990636),
 ('saw', 0.6074949524235931)]
[('movie', 1.0000000000000002),
 ('movies', 0.6264000901632075),
 ('waste', 0.5902814862708345),
 ('thought', 0.5603982416345786),
 ('watch', 0.5486201326296053)]
[('film', 1.0),
 ('films', 0.6104541571876486),
 ('cinema', 0.5324352048374879),
 ('director', 0.5139937670502374),
 ('cinematography', 0.5138508962007942)]
```

Monolingual corpus, the least frequent words:

```
[('according', 0.9999999999999999),
 ('everything', 0.9931032867180424),
 ('place', 0.9928814512460278),
 ('says', 0.9926084092220174),
 ('keep', 0.9923692240019467)]
[('serve', 1.0),
 ('example', 0.9884668620184874),
 ('appears', 0.9881421908147624),
 ('seems', 0.9881201223258333),
 ('nature', 0.9880742693529966)]
[('breaking', 1.0),
 ('work', 0.9889140793499385),
 ('taken', 0.9885708775893547),
 ('today', 0.9884574294589742),
 ('number', 0.9882349708414532)]
[('favorites', 1.0),
 ('watching', 0.9783481244248106),
 ('loved', 0.9778277195482216),
 ('opinion', 0.9771952444770546),
 ('enjoyed', 0.9770937336798221)]
[('mistakes', 1.0),
 ('means', 0.9868802609693922),
 ('makes', 0.986564466448231),
 ('miss', 0.9861086700731426),
 ('kind', 0.9859963407857474)]
```
Parallel crorpus, most frequent words:
```
[('de', 1.0000000000000002),
 ('multi-people', 0.9999899114091048),
 ('pan-alpine', 0.9999899114091048),
 ('growth-promoter', 0.9999899114091048),
 ('pre-embarkation', 0.9999899114091048)]
[(',', 1.0),
 ('gatherers', 0.9999021163470986),
 ('supplies.', 0.9999021163470986),
 ('double-breasted', 0.9999021163470986),
 ('polanyi', 0.9999021163470986)]
[('a', 1.0),
 ('an', 0.998983395138381),
 ('one-country', 0.9623823694665461),
 ('one-month', 0.9149363845284196),
 ('break-neck', 0.8491210590663577)]
[('se', 0.9999999999999999),
 ('self-assesses', 0.8328265125462805),
 ('self-correct', 0.8328265125462805),
 ('self-responsible', 0.8328265125462805),
 ('self-understanding', 0.8328265125462805)]
[('la', 0.9999999999999998),
 ('superscription', 0.9999586464999254),
 ('frau', 0.9999586464999254),
 ('consorted', 0.9999586464999254),
 ('orients', 0.9999586464999254)]
```
- **Is there a difference in the quality of the output for the high-frequency versus the low-frequency words?**  
Yes, there is a significant difference in the quality of the output when it comes to the frequency of the target word, though considering the monolingual corpus only (due to the issue with the parallel data stated below). In general, more meaningful relations were extracted with the more frequent words, while the output of the least frequent ones is somewhat chaotic.   
- **Is there a difference for the two types of contexts used?**  
Yes, there is a difference. Given that the parallel corpus was not POS-tagged due to the reported memory error, we could not filter content words and remove some noise. Therefore, the results for the parallel data are much worse than expected and uncomparable to the monolingual ones.  

- **Can you explain what you see?**
In the monolingual corpus, the five most frequent words are *is, was, movie, film* and *‘s* (which was not taken into account in the further analysis). Given that the corpus contains movie reviews, this output is not surprising. The most similar words found for the two verbs are mostly other verbs, except the noun *man* that was extracted as similar with the verb *is*. The most interesting result is obtained with the word *film* that got as the most similar words other nouns very related to the topic of movie reviews: *cinema, director, cinematography*. When it comes to the output of the least frequent words, there is not much regularity in the output: similar words are extracted from all types of part-of-speech and most of the time there is not much connection between the target word and the extracted similar words.    
&ensp;&ensp; When it comes to the parallel corpus, as the most frequent words we obtained only stop words (*de, , , la, se, a*) that were aligned with all sorts of words without visible regularity. The least frequent word list was not extracted, since the results would have been equally bad.  
&ensp;&ensp; Perhaps better results would be obtained if the corpora were lemmatized since in that way we would not have the same word appearing in its different forms in the same output (for example, the forms *is, are, was, were*). Also, it would have been beneficial if we managed to POS-tag the parallel data as well, since in that way we would be able to filter the stop words and to some extent remove noise.

#### 13) Explain the notion of contextual variability. (5 points)
Take a polysemous word of your choice that is among the 1550 head words you selected. Generate the top-10 semantically similar words for this target word using the monolingual DSM you created. Paste the output in the report.  
**- Do you see some consequences of contextual variability in the outputs for this word?**  
Now take the multilingual DSM. Find a polysemous word in the features of the multilingual DSM1. For its translation in your target language, find the top-10.  
**- What are your observations?**  
**- Do you think using multiple languages could have improved the multilingual DSM? **  
We take as an example the word 'get'. From our monolingual corpus we get the following similar words:

```
[('get', 1.0),
 ('everyone', 0.9964144093340506),
 ('mind', 0.9962880682030615),
 ('believe', 0.9962865266548062),
 ('come', 0.9962714101107594),
 ('give', 0.9962699881148831),
 ('reason', 0.9962526357871062),
 ('thing', 0.9962040639305845),
 ('look', 0.9961682074477383),
 ('things', 0.9960742493531183),
 ('end', 0.9960663558282294),
 ('take', 0.9960522935950892),
 ('anything', 0.9960490497511368),
 ('something', 0.9960261419004385),
 ('ca', 0.9959793968425902)]
 ```
 
 Here we can see that 'get' has different meanings depending on its context: we can imagine that something like 'lets get everyone together' means 'put together', while when 'look' is near 'get', then it probably builds 'get a look' which means 'see': this phenomena can be described in terms of contextual variability.
 
Note: we are not reporting on the second part of this task, since we are getting strange results in extracting words in the opposite direction: for instance, when trying to get the neighbours of 'tinto' (i.e. 'red') we get the following:

```
[('tinto', 1.0),
 ('red', 0.057016996594512454),
 ('out', 0.0),
 ('euro-apathy', 0.0),
 ('mrs', 0.0),
 ('danger', 0.0),
 ('discouraging', 0.0),
 ('allow', 0.0),
 ('settings', 0.0),
 ('readjustments', 0.0),
 ('transposing', 0.0),
 ('inter-pillar', 0.0),
 ('achieve', 0.0),
 ('securing', 0.0),
 ('willing', 0.0)]
```

We suspect that there is a bug in the pipeline, but at this point we were not able to detect it.

------------------
#### *Compositionality:*

#### 14) Explain the principle of compositionality and discuss a couple of different ways in which DSMs have been adapted to cater for compositionality. (3 points)
Most of the work done in Distributional semantics takes into account only single, isolated words (Guevara, 2011). However, in language we combine words into bigger structures, phrases and sentences, such as *strong coffee, table cloth, hot dog*, etc. (Van der Plas, 2017). Therefore, we would want DSMs to be able to deal with these more complex structures as well. This is where the compositionality theory comes in place, which is the fundamental idea in most of the contemporary works in semantics (Van der Plas, 2017). It is based on the Principle of compositionality: the meaning of a complex expression is comprised of the meaning of its parts and by the syntactic rules that combine them (Partee, ter Meulen and Wall, 1990 in Guevara, 2011).  
&ensp;&ensp; So the question rises how to deal with compositionality in DSMs. This is not an easy task and if we take a look at the compounds as a quite good example of how problematic compositionality is, we will see that there are several levels of compositionality (Van der Plas, 2017). Some compounds can have somewhat  compositional meaning, as in *table cloth*, others can have it partially compositional as *data base*, while some will have a meaning that cannot be split into meaningful parts as in *hot dog*. Besides, they could consist of combinations of different parts of speech, such as ADJ-N, N-N, V-N, etc. (ibid.).    
&ensp;&ensp; Previous research has had several ideas to tackle the problem of compositionality using DSMs models. Firstly, the attempt was made to treat complex expressions as a whole unit, since some of them already are merged together as for example the word *database* or compounds in general in German language. Following the Distributional hypothesis, we could then represent the meaning of a compound by looking at all the contexts it appears in. However, the problem with this approach is that it works only when looking into a really large corpus of text and with only very frequent compounds. It is clear that data sparsity would be an issue since the rare compounds would be quite badly represented (Van der Plas, 2017). Besides, even with the frequent compounds there an issue of being able to generalize or reuse word meaning in a more transparent use. (ibid.).  
&ensp;&ensp; Given all the issues previously stated, the work done after this treats compositionality as a function, which means that given two independent vectors, the semantically compositional, resulting vector is created by one of the following operations: vector addition, (pointwise) vector multiplication or tensor product (Guevara, 2011; Van der Plas, 2017).  Vector addition is one of the most common approaches in the literature introduced by Mitchell and Lapata (2010) (as stated in Van der Plas, 2017) and it involves a simple addition as given in the example 8 below.   

(8)  summation: v1 + v2 = v3  
&ensp;&ensp; red \[1 2 0\] +  house  \[14 16 20\] =  red house \[15 18 20\]  
<br>
&ensp;&ensp; multiplication: v1 x v2 = v3  
&ensp;&ensp; red \[1 2 0\] x  house  \[14 16 20\] = red house \[14 32 20\]  
<br>
&ensp;&ensp; tensor product: v1⊗v2 = v3  
&ensp;&ensp; red = \[1 2 0\] ⊗ house = \[0 1 3\] = red house \[1\*0, 1\*1, 1\*3, 2\*0, 2\*1, 2\*3, 0\*0, 0\*1, 0\*3\] =  red house \[0, 1, 3, 0, 2, 6, 0, 0, 0\]   
<br>
So given two vectors v1 and v2, we get a compositional meaning of v3 by summing the previous two up. Even though summation of vectors is computationally efficient, it does not provide us with an ideal solution (Van der Plas, 2017): the new vector is in between the two, but is more governed by the one that contains higher individual frequency counts. In the example of the *red house*, the new vector is more influenced by the vector *house* than the vector *red*. Therefore, it resembles more a union than an intersection (ibid.).  
&ensp;&ensp; Mitchell and Lapata (2010) introduced also pointwise-multiplication of vectors. It differs from the addition model by only taking into account non-zero components of the vectors while, as seen previously, the summation considers all of the components. The intuition is similar, we multiply two vectors and get as an output a third one that should correspond to the meaning of a complex expression.    
&ensp;&ensp; Widdows (2008) (in Guevara, 2011) introduces a more complex vector operation, tensor product, in order to capture meaning compositionality. The result of a tensor product of two vectors is a high dimensional matrix, which was shown empirically to catch better the compositionality of meaning (Van der Plas, 2017).    
&ensp;&ensp; All of the previously mentioned approaches have one thing in common: they all apply one single geometric operation on the vectors v1 and v2. However, it seems unlikely that only one operation can work on all semantic and syntactic relations and in all languages (Guevara, 2011).

#### 15) Now, let us experiment with compositionality in the monolingual DSM you have just built. Follow the instructions on http://clic.cimec.unitn.it/composes/toolkit/composing.html and select a model of your choice (motivate your choice). Show some example output of the composed DSM. Analyse the results. (5 points)

#### 16) Is it possible to create a compositional model for the multilingual DSM with the models you have chosen? If so, show some example output here as well. Analyse the results. (5 points)

For the practical part of the compositionality task we decided to do something different from what it was asked in questions 15 and 16: DISSECT is based on research that was carried on prior to the huge spike of popularity of word embeddings and neural networks and here we want to demonstrate the compositionality properties of these models. We will show how to solve an analogy problem using embeddings created using GloVe. The task is the following:
<br>  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; **man:king=woman:?**  
<br>
We have to complete this proportion: 'man stays to king as woman stays to what' ? In order to answer this question, we represented these words as a point in a dense high-dimensional space: this is the space created when training the embeddings on millions of texts. In this case, the corpus is the CommonCrawl.  
&ensp;&ensp; The assumption of the vector space models of meaning is that meaning is a point in space: by leveraging the arithmetic properties of the vectors that populate a space, we can move from one point to another. In this case, we start from `king`, we subtract the `man` part of its meaning and we add `woman`. The code below shows the result.

In [ ]:
from numpy import dot
from numpy.linalg import norm
import spacy

cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

nlp = spacy.load('en_core_web_lg')

king = nlp.vocab[u'king']
man = nlp.vocab[u'man']
woman = nlp.vocab[u'woman']

result = king.vector - man.vector + woman.vector

vocabulary = [w for w in nlp.vocab if w.has_vector and w.orth_.islower() and w.lower_ not in ['king','man','woman']]

vocabulary.sort(key=lambda w: cosine(w.vector, result))
vocabulary.reverse()

print(vocabulary[0].orth_)

===RESULT===
queen

------------------------
#### *Relating things back to theory:*

**We discussed several theories in class that try to explain the lexical semantics of words. We are now trying to couple our knowledge from one of these theories to the things we observe in the DSM. 1 For example, if English is your target language and you chose German as the counterpart in the parallel text, you could choose Kiefer in German (see slides). One of its translations in English is pine. Now, get the top-10 most similar words of pine in the multilingual DSM and describe if you see any unexpected nearest neighbours. You can try with several polysemous words, if you like.**

#### 17) Discuss the basic tenet of the Prototype Theory in lexical semantics and discuss at least two advantages when compared to the classical theory. (5 points)
The Prototype Theory emerged as a reaction to the problem with psychological data in the Classical theory providing an alternative approach (Laurence and Margolis, 1999). Even though there is not one single prototype theory all theorists agree about, the general idea is the same: “most concepts - including most lexical concepts - are complex representations whose structure encodes a statistical analysis of the properties their members tend to have” (ibid.). In other words, most items in the extension of a concept will have kind of features “typical” of a concept. However, there will be some items that will not be represented by all of the “typical” features and yet still belong to the same concept. This points to the fact that not all features are by default necessary and that some of them are more important than others (Van der Plas, 2017). This is different than what the Classical theory suggests - that all features are necessary and that an item has to satisfy all of features in order to belong under a certain concept (Laurence and Margolis, 1999). On the contrary, the Prototype theory suggests satisfying a sufficient number of features out of which some carry more importance than others (ibid.). For example, if *bird* is composed by features such as *flies, sings, has feathers, lays eggs, nests in trees*, etc. then we could say that a *sparrow* is a bird because it has all of the features. The problem in the Classical theory would come up with birds such as *ostrich* since it does not have all the features and therefore would not be considered a bird. The Prototype theory takes care of these kind of examples because even though *ostrich* does not have all the features, it has enough to be considered a bird -  they do not fly, but they have feathers, lay eggs, etc. An item that satisfies all the features, or most of them, could be considered a prototypical example of a concept.  
&ensp;&ensp; The previously stated rejection of the idea about “necessary and sufficient conditions” goes along with Wittgenstein's idea that things that fall under the same concept form "a complicated network of similarities overlapping and crisscrossing: sometimes overall similarities, sometimes similarities of detail" (Laurence and Margolis, 1999), in other words they have a “family resemblance”. Rosch and Mervis (1975, as stated in Laurence and Margolis, 1999) explain this further: “formal criteria are neither a logical nor psychological necessity; the categorical relationship in categories which do not appear to possess criterial attributes \[...\] can be understood in terms of the principle of family resemblance.” In other words, they all state that concepts are not governed by definitions but by a unfixed set of properties that will appear in different combinations with different items under a certain concept. The example they give is *game*; some games will have one set of properties, some will have other. No matter what exact set of properties they have, what makes them similar and therefore fall under the same concept is that their properties overlap in a way that they create a similarity space (Laurence and Margolis, 1999). So what makes a certain game fall under the concept *game* is that it is within boundaries of this similarity space (ibid.).  
&ensp;&ensp; Since the Prototype theory was created mostly as a response to the classical theory, it solves some of its problems, one of which was mentioned before about family resemblance and no need for a definition of concepts. This lack of definitions is the problem the Classical theory had that the Prototype theory successfully tackled by claiming and proving that concepts do not have a definitional structure (Laurence and Margolis, 1999). Second advantage the Prototype theory has over the Classical theory is its treatment of categorization, which tackles the issues the latter theory had: conceptual fuzziness and typicality effect (ibid.). Conceptual fuzziness refers to the problem of many concepts being inexact (as for example a question of *carpet* belonging or not to the concept of *furniture), while the typicality effects refer to the problems where we have items that are not typical examples of a concept and therefore do not have all the features it could have, but still pertain to a certain concept (question of a more typical *fruit*: *apple* or *olive*) (Van der Plas, 2017). The Prototype theory puts into a certain category an instance if the representation of that instance is sufficiently similar to the representation of the category (Laurence and Margolis, 1999). In other words, it compares an instance with a prototypical example of the category and if there is enough feature overlap, then they are similar and belong to the same category. For this purpose, they have developed different measures of similarity: they compare all the features in parallel and depending on a feature, it can add either a positive or a negative value to the “calculation” depending if it is a common feature or not. If the result obtained is not high enough to make a decision about whether an instance belongs or not to a certain category, they just do not make a judgement. And in their idea, this solves the fuzziness problem, they just do not make any positive or negative judgment if there is not enough evidence, and agree with the fact that some instances can be somewhere in between (Laurence and Margolis, 1999).

#### 18) What are typicality effects? Does the DSM cater for typicality effects? (5 points)
The typicality effect refers to the fact that not all the members of a certain category are equally typical examples of that category (Laurence and Margolis, 1999; Connell and Ramscar, 2001). As stated in Laurence and Margolis (1999), this was first proven by Rosch (1973) who found that the participants in the study were able to rank items as more or less typical members of a certain category. The example given to the participants was to sort different types of fruits (*apple, fig, plum, olive*) on the 7-point scale by how well they represent the category *fruit*. This showed that certain fruits are commonly addressed as typical (*apple*), while others were commonly marked as atypical (*olive*) (ibid.).  
&ensp;&ensp; If we take a look at the distributional similarity scores of different words for fruit and the concept *fruit* in the table 6, we will notice that there are some significant differences. We will see that for the more typical fruits chosen by participants in the above mentioned study (*apple, plum*) we get similar distributional results, while for the *olive* (atypical example), we get a completely different set of similar words (that we could argue share more similar properties with the olive, as for example “high in fat”). And also, we see that *apple* does appear as one of the typical neighbours of *fruit* while *olive* does not. Since the distributional similarity depends on the context these words appear in, we could say that the more typical examples of fruits tend to appear in different types of context than the atypical ones.     

|       apple      |       plum       |      olive        |      fruit      |
|:-----------------|:-----------------|:------------------|:----------------|  
| strawberry 0.808 | cherry 0.806     | olive 0.819       | berry 0.83      |
| pear 0.793       | pear 0.770       | almond 0.69       | mango 0.71      |
| cherry 0.780     | apricot 0.769    | olive 0.663       | vegetable 0.71  |
| peach 0.760      | apple 0.746      | carob 0.652       | drupe 0.70      |
| plum 0.746       | strawberry 0.739 | apricot 0.650     | flower 0.69     |
| apricot 0.744    | peach 0.736      | pistachio 0.631   | apple 0.69      |
| almond 0.740     | quince 0.735     | citrus 0.628      | strawberry 0.68 | 
| quince 0.707     | persimmon 0.734  | avocado 0.627     | pear 0.68       |
| watermelon 0.705 | raspberry 0.705  | cinnamon 0.623    | apricot 0.68    |
| mango 0.694      | blackberry 0.699 | pomegranate 0.616 | pome 0.68       |

<h6 style="text-align: center;" markdown="1">Table 6. Similarity scores among 10 nearest neighbours of words under the concept "fruit". </h6>  
<br>
\* Similarity scores in the table 6 are obtained from Wikipedia from [this](http://vectors.nlpl.eu/explore/embeddings/en/#) page.  

-------------------
### Team work distribution by assignment questions:

- **Angelo**:  
1, 4, 5, 7, 11, 13, 14

- **Jovana**:  
2, 3, 6, 8, 10, 12, 17 

- **Together:**  
9, 18, man:king=woman:?


--------------------
### References

- Basili, Roberto and Marco Pennacchiotti. (2010). Distributional lexical semantics: Toward uniform representation paradigms for advanced acquisition and processing tasks. In: Natural Language Engineering, 16 (4), pp. 347–358, 2010.  
<br>
- Connell, Louise and Michael Ramscar. (2001). Using Distributional Measures to Model Typicality in Categorization. *In Proceedings of the 23rd Annual Conference of the Cognitive Science Society*. University of Edinburgh.  
<br>
- Evert, Stefan (2008). Corpora and collocations. *In A. Lüdeling and M. Kytö (eds.), Corpus Linguistics. An International Handbook*, chapter 58. Mouton de Gruyter, Berlin.  
<br>
- Evert, Stefan, Marco Baroni, and Alessandro Lenci. (2010). *Distributional Semantic Models*, Tutorial at NAACL-HLT 2010, Los Angeles, CA.  
<br>
- Guevara, Emiliano. (2011). Computing semantic compositionality in distributional semantics. *In Proceedings of the Ninth International Conference on Computational Semantics (IWCS '11)*. Association for Computational Linguistics, Stroudsburg, PA, USA, pp. 135-144.  
<br>
- Harris, Zellig S. (1968). *Mathematical structures of language*. New York: Interscience Publishers.  
<br>
- Jurafsky, Daniel and James H. Martin. (2017). Speech and Language Processing. Draft from August 2017, available [here](https://web.stanford.edu/~jurafsky/slp3/15.pdf).  
<br>
- Laurence, Stephen and Eric Margolis. (1999). Concepts and Cognitive Science. *In Margolis, Eric and Stephen Laurence (eds.) Concepts: Core Readings*, pp. 3-81. Cambridge, Mass.: MIT Press.  
<br>
- Miller, George A., Richard Beckwith, Christiane Fellbaum, Derek Gross, and Katherine Miller. (1990). Introduction to WordNet: an on-line lexical database. *International Journal of Lexicography* 3, (4), pp. 235-244, 1990.  
<br>
- Morato, Jorge, Miguel Ángel Marzal, Juan Lloréns, and José Moreiro. (2004). WordNet Applications. *In Proceedings of GWC*, pp. 20-23, 2004.  
<br>
- Sahlgren, Magnus. (2001). The distributional hypothesis. *Italian Journal of Linguistics* 20, pp. 33-53, 2001.  
<br>
- Turney, Peter D. and Patrick Pantel. (2010). From frequency to meaning: vector space models of semantics. *Journal of Artificial Intelligence Research* 37, (1), pp.141-188, 2010.  
<br>
- Van der Plas, Lonneke. (2017). *LIN2580/LIN5580: Computational Lexical Semantics, week 6 notes* \[PowerPoint slides\]. Retrieved from [here](https://www.um.edu.mt/vle/pluginfile.php/272004/mod_resource/content/5/CompLexSemLect6.pdf).  